In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import pandas as pd

from matplotlib import font_manager, rc

import folium
import branca
from shapely.geometry.point import Point

from utils import round, cal_area, generate_candidate_sites
from mclp import mclp

In [ ]:
jaeseorham = pd.read_csv('../data/seoul_jaeseorham.csv', encoding='cp949')
jaeseorham = jaeseorham[jaeseorham['관리기관명'] == '은평구']

grand_medical = pd.read_csv('../data/grand_medical_facility.csv', encoding='cp949')
child = pd.read_csv('../data/child_safe_site.csv', encoding='cp949')
freezing = pd.read_csv('../data/org/도로교통공단_결빙사고_다발지역_은평구.csv')
warnway = pd.read_csv('../data/13_20_freezing.csv', encoding='cp949')

In [ ]:
radius = 208 #1개의 제설함 커버 반경
y_u = round(lat = "lat", lon = "lon", dataset=jaeseorham, r=radius) #제설함
gm_u = round(lat = "lat", lon = "lon", dataset=grand_medical) #노인 복지시설
ch_u = round(lat = "lat", lon = "lon", dataset=child, r=150) #어린이 보호구역 스쿨존 300m
fr_u = round(lat = "la_crd", lon = "lo_crd", dataset=freezing) #결빙
ww_u = round(lat = "lat", lon = "lon", dataset=warnway) #급경사지

In [ ]:
lat_c, lon_c = '37.609518', '126.926149' # 서울 중심 위/경도

m = folium.Map([lat_c, lon_c], zoom_start = 13)# 현재 제설함은 보라색, 필요한 구역 빨간색
for lat, lon in zip(np.array(jaeseorham['lat']), np.array(jaeseorham['lon'])):
    folium.Circle(location = [lat, lon], radius = 208, color='black', fill_color = 'purple', weight=1).add_to(m)
for lat, lon in zip(np.array(grand_medical['lat']), np.array(grand_medical['lon'])):
    folium.Circle(location = [lat, lon], radius = 50, color='black', fill_color = 'red', weight=1).add_to(m)
for lat, lon in zip(np.array(child['lat']), np.array(child['lon'])):
    folium.Circle(location = [lat, lon], radius = 50, color='black', fill_color = 'red', weight=1).add_to(m)
for lat, lon in zip(np.array(freezing['lat']), np.array(freezing['lon'])):
    folium.Circle(location = [lat, lon], radius = 50, color='black', fill_color = 'red', weight=1).add_to(m)
for lat, lon in zip(np.array(warnway['lat']), np.array(warnway['lon'])):
    folium.Circle(location = [lat, lon], radius = 50, color='black', fill_color = 'red', weight=1).add_to(m)
m

In [ ]:
nlsp = gpd.read_file('../data/nlsp_021001001.shp',encoding = 'utf8').to_crs(epsg=4326)

In [ ]:
m2 = folium.Map([lat_c, lon_c], zoom_start = 13)
for _, row in nlsp.iterrows():
    value = 1 - cal_area(row['geometry'], y_u)
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    color = plt.cm.OrRd(value)
    color = mpl.colors.to_hex(color)
    
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m2)

colormap = branca.colormap.linear.OrRd_06.scale(0, 1)
colormap = colormap.to_step(index=np.linspace(0, 1, 1000))
colormap.caption = 'Area Ratio NOT Covered by Jae-seor-ham'
colormap.add_to(m2)
m2

**$w_i = lbl \times AreaRatio \times child \times grand medical \times freezing \times warnway$** 

- $w_i$ : 수요량
- $lbl$ : 주거 인구 비례
- $AreaRatio$ : 커버되지않은 면적 비율 (0~1)
- $child$ : 어린이 보호 구역
- $grandmedical$ : 노인 의료 복지 시설
- $freezing$ : 결빙 사고 다발 지역
- $warnway$ : 급경사지

In [ ]:
m2_nlsp = nlsp.replace(to_replace = 'N/A', value = 0)
m2_nlsp = m2_nlsp.astype({'lbl':float})

In [ ]:
m3 = folium.Map([lat_c, lon_c], zoom_start = 13)
values = []
cnt = 0
for _, row in m2_nlsp.iterrows():
    value = (1 - cal_area(row['geometry'], y_u)) * (row['lbl']) * (1 + cal_area(row['geometry'], ch_u)) * (1 + cal_area(row['geometry'], gm_u))
    if value >= 0.1:
        cnt += 1
    else:
        value = 0
    values.append(value)
max_value = max(values)

for i, row in m2_nlsp.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.Reds(value/max_value)
    color = mpl.colors.to_hex(color)

    if value == 0:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0,
                                                                    })
    else:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m3)
    
colormap = branca.colormap.linear.Reds_05.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1500))
colormap.caption = 'lbl × Area Ratio not covered by jae-seor-ham'
colormap.add_to(m3)
m3

In [ ]:
print(f'max_value : {max_value:.4f}')
print(f'N_Grid(value>0) : {cnt}, N_Grid(All) : {len(m2_nlsp)}')
print(f'Grid_Ratio : {cnt/len(m2_nlsp):.4f}')

In [ ]:
m2_nlsp['weight'] = values
m2_nlsp['geo'] = nlsp['geometry'].to_crs(epsg=5179)

In [ ]:
M = cnt
K = 0
l = len(m2_nlsp)

new_df = pd.DataFrame(data={'K':[0], 'max_value':[np.round(max_value, 4)], 'mean_value':[np.mean(m2_nlsp['weight'])],
                            'N_Grid(value>0)':cnt, 'Grid_Ratio':np.round(cnt/l,4)})
new_df['K'] = new_df['K'].astype(int)
while True:
    K += 1
    opt_sites_org, f = mclp(m2_nlsp, K, M, radius)
    
    opt_df= pd.DataFrame(opt_sites_org, columns=['lon', 'lat'])
    opt_df['geom'] = opt_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    gdf = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
    gdf = gdf.to_crs('epsg:4326')
    gdf['lon'] = gdf['geom'].apply(lambda p:p.x)
    gdf['lat'] = gdf['geom'].apply(lambda p:p.y)
    lats = gdf['lat']
    lons = gdf['lon']
    opt_df = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
    opt_df['geometry'] = opt_df.geom.buffer(radius)
    gdf = opt_df.to_crs('epsg:4326')

    u_tmp = y_u
    for c in gdf['circle']:
        u_tmp = u_tmp.union(c)
    
    values = []
    for _, row in m2_nlsp.iterrows():
        value = (1 - cal_area(row['geometry'], u_tmp)) * row['weight']
        if value < 0.1:
            value = 0
        values.append(value)
    max_value = max(values)
    
    tmp_df = pd.DataFrame({'K':[int(K)], 'max_value':[np.round(max_value, 4)], 'mean_value':[np.mean(values)],
                        'N_Grid(value>0)':[cnt-f], 'Grid_Ratio':[np.round((cnt-f)/l,4)]},
                         index=[K])
    new_df = pd.concat([new_df, tmp_df])

    if f == M:
        break

print(f'최소 {K}개의 제설함으로 모든 취약 지점을 커버할 수 있다.')
new_df

In [ ]:
m4 = folium.Map([lat_c, lon_c], zoom_start = 13)
values = m2_nlsp['weight']
max_value = max(values)
for i, row in m2_nlsp.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.Reds(value/max_value)
    color = mpl.colors.to_hex(color)

    if value == 0:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0,
                                                                    })
    else:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m4)
colormap = branca.colormap.linear.Reds_05.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1500))
colormap.caption = 'lbl × Area Ratio not covered by jae-seor-ham'
colormap.add_to(m4)
for lat, lon in zip(lats, lons):
    folium.Circle(location = [lat, lon], radius = radius, color='black', fill_color = 'blue', weight=1).add_to(m4)
m4.save('mclp.html')
m4

In [ ]:
K = 24 # 설치할 갯수
M = cnt
opt_sites_org, f = mclp(m2_nlsp, K, M, radius)
print('커버되는 제설함 수요 지점의 개수 : {}'.format(f))

opt_df= pd.DataFrame(opt_sites_org, columns=['lon', 'lat'])
opt_df['geom'] = opt_df.apply(lambda r: Point(r['lon'], r['lat']), axis=1)
gdf = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
gdf = gdf.to_crs(epsg=4329)
gdf['lon'] = gdf['geom'].apply(lambda p:p.x)
gdf['lat'] = gdf['geom'].apply(lambda p:p.y)

m5 = folium.Map([lat_c, lon_c], zoom_start = 13)
for lat, lon in zip(gdf['lat'], gdf['lon']):
    folium.Circle(location = [lat, lon], radius = radius, color='black', fill_color = 'blue', weight=1).add_to(m4)
m5